In [1]:
import math
import random
import warnings
from typing import Literal, NamedTuple
import pandas as pd
import numpy as np
import rtdl_num_embeddings  
import scipy.special
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing
import torch
import torch.nn.functional as F
import torch.optim
from torch import Tensor
from tqdm.std import tqdm
from pandas import read_csv
import os
warnings.simplefilter('ignore')
from tabm_reference import Model, make_parameter_groups
warnings.resetwarnings()

## prepare data


In [2]:
dataset = pd.read_excel(r'D:\Data\NHRI\NHDF\Train_set.xlsx')
print(type(dataset))
dataset.head()

<class 'pandas.core.frame.DataFrame'>


,门宽,门高,门厚,门后压力,上倾,下倾,对称倾角,开度,倾角正弦值,水头,底缘单宽净动水荷载
0,5.4,5.2,0.95,4,1,0,0,0.1,0.300706,17.2,75.776301
1,5.4,5.2,0.95,4,1,0,0,0.1,0.300706,20.0,94.489334
2,5.4,5.2,0.95,4,1,0,0,0.1,0.300706,22.5,106.259588
3,5.4,5.2,0.95,4,1,0,0,0.1,0.300706,25.0,122.277321
4,5.4,5.2,0.95,4,1,0,0,0.1,0.300706,28.2,134.698419


In [3]:
values = dataset.values
values = values.astype('float32')
print(type(values))
#print(values[0:5])
values = np.array(values)
print(values[0:5])
print(values.shape)

<class 'numpy.ndarray'>
[[5.4000001e+00 5.1999998e+00 9.4999999e-01 4.0000000e+00 1.0000000e+00
  0.0000000e+00 0.0000000e+00 1.0000000e-01 3.0070579e-01 1.7200001e+01
  7.5776299e+01]
 [5.4000001e+00 5.1999998e+00 9.4999999e-01 4.0000000e+00 1.0000000e+00
  0.0000000e+00 0.0000000e+00 1.0000000e-01 3.0070579e-01 2.0000000e+01
  9.4489334e+01]
 [5.4000001e+00 5.1999998e+00 9.4999999e-01 4.0000000e+00 1.0000000e+00
  0.0000000e+00 0.0000000e+00 1.0000000e-01 3.0070579e-01 2.2500000e+01
  1.0625959e+02]
 [5.4000001e+00 5.1999998e+00 9.4999999e-01 4.0000000e+00 1.0000000e+00
  0.0000000e+00 0.0000000e+00 1.0000000e-01 3.0070579e-01 2.5000000e+01
  1.2227732e+02]
 [5.4000001e+00 5.1999998e+00 9.4999999e-01 4.0000000e+00 1.0000000e+00
  0.0000000e+00 0.0000000e+00 1.0000000e-01 3.0070579e-01 2.8200001e+01
  1.3469843e+02]]
(600, 11)


In [4]:
input_features = 10
output_features = 1

X_train = values[:, 0 : input_features]
y_train = values[:, -1]
print(X_train.shape)
print(y_train.shape)

print(type(X_train))
print(type(y_train))

print("X_train.shape: ", X_train.shape)
print("y_train.shape: ", y_train.shape)

(600, 10)
(600,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
X_train.shape:  (600, 10)
y_train.shape:  (600,)


In [5]:
# output name
label = '底缘单宽净动水荷载'

# NOTE: read all test sets
test_data = read_csv(r'D:\Data\NHRI\NHDF\Test_set.csv', encoding = 'utf-8') #, header = None
y_test = test_data[label]
x_test = test_data.drop(columns=[label])
print(type(y_test))
print(type(x_test))
y_test = np.array(y_test)
x_test = np.array(x_test)
print(type(y_test))
print(type(x_test))
print(y_test.shape)
print(x_test.shape)

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(72,)
(72, 10)


In [6]:
# NOTE: read 1st test set
test_data = read_csv(r'D:\Data\NHRI\NHDF\1st_test.csv', encoding = 'utf-8') #, header = None
y_test1 = test_data[label]
x_test1 = test_data.drop(columns=[label])
y_test1 = np.array(y_test1)
x_test1 = np.array(x_test1)
print(y_test1.shape)
print(x_test1.shape)

# NOTE: read 2nd test set
test_data = read_csv(r'D:\Data\NHRI\NHDF\2st_test.csv', encoding = 'utf-8') #, header = None
y_test2 = test_data[label]
x_test2 = test_data.drop(columns=[label])
y_test2 = np.array(y_test2)
x_test2 = np.array(x_test2)
print(y_test2.shape)
print(x_test2.shape)

# NOTE: read 3rd test set
test_data = read_csv(r'D:\Data\NHRI\NHDF\3st_test.csv', encoding = 'utf-8') #, header = None
y_test3 = test_data[label]
x_test3 = test_data.drop(columns=[label])
y_test3 = np.array(y_test3)
x_test3 = np.array(x_test3)
print(y_test3.shape)
print(x_test3.shape)

# NOTE: read 4th test set
test_data = read_csv(r'D:\Data\NHRI\NHDF\4st_test.csv', encoding = 'utf-8') #, header = None
y_test4 = test_data[label]
x_test4 = test_data.drop(columns=[label])
y_test4 = np.array(y_test4)
x_test4 = np.array(x_test4)
print(y_test4.shape)
print(x_test4.shape)

# NOTE: read 5th test set
test_data = read_csv(r'D:\Data\NHRI\NHDF\5st_test.csv', encoding = 'utf-8') #, header = None
y_test5 = test_data[label]
x_test5 = test_data.drop(columns=[label])
y_test5 = np.array(y_test5)
x_test5 = np.array(x_test5)
print(y_test5.shape)
print(x_test5.shape)

# NOTE: read 6th test set
test_data = read_csv(r'D:\Data\NHRI\NHDF\6st_test.csv', encoding = 'utf-8') #, header = None
y_test6 = test_data[label]
x_test6 = test_data.drop(columns=[label])
y_test6 = np.array(y_test6)
x_test6 = np.array(x_test6)
print(y_test6.shape)
print(x_test6.shape)

# NOTE: read 7th test set
test_data = read_csv(r'D:\Data\NHRI\NHDF\7st_test.csv', encoding = 'utf-8') #, header = None
y_test7 = test_data[label]
x_test7 = test_data.drop(columns=[label])
y_test7 = np.array(y_test7)
x_test7 = np.array(x_test7)
print(y_test7.shape)
print(x_test7.shape)

# NOTE: read 8th test set
test_data = read_csv(r'D:\Data\NHRI\NHDF\8st_test.csv', encoding = 'utf-8') #, header = None
y_test8 = test_data[label]
x_test8 = test_data.drop(columns=[label])
y_test8 = np.array(y_test8)
x_test8 = np.array(x_test8)
print(y_test8.shape)
print(x_test8.shape)

# NOTE: read 9th test set
test_data = read_csv(r'D:\Data\NHRI\NHDF\9st_test.csv', encoding = 'utf-8') #, header = None
y_test9 = test_data[label]
x_test9 = test_data.drop(columns=[label])
y_test9 = np.array(y_test9)
x_test9 = np.array(x_test9)
print(y_test9.shape)
print(x_test9.shape)

(8,)
(8, 10)
(8,)
(8, 10)
(8,)
(8, 10)
(8,)
(8, 10)
(8,)
(8, 10)
(8,)
(8, 10)
(8,)
(8, 10)
(8,)
(8, 10)
(8,)
(8, 10)


## build dataset


In [8]:
X_cont: np.ndarray = X_train
Y: np.ndarray = y_train

X_cont: np.ndarray = X_cont.astype(np.float32)
n_cont_features = X_cont.shape[1]
Y = Y.astype(np.float32)

# >>> Split the dataset.
all_idx = np.arange(len(Y))
train_idx, val_idx = sklearn.model_selection.train_test_split(  # randomly split train and test set
    all_idx, train_size=0.8
)

data_numpy = {
    'train': {'x_cont': X_cont[train_idx], 'y': Y[train_idx]},
    'val': {'x_cont': X_cont[val_idx], 'y': Y[val_idx]},
    'test': {'x_cont': x_test, 'y': y_test},     
    'test1': {'x_cont': x_test1, 'y': y_test1},
    'test2': {'x_cont': x_test2, 'y': y_test2},
    'test3': {'x_cont': x_test3, 'y': y_test3},
    'test4': {'x_cont': x_test4, 'y': y_test4},
    'test5': {'x_cont': x_test5, 'y': y_test5},
    'test6': {'x_cont': x_test6, 'y': y_test6},
    'test7': {'x_cont': x_test7, 'y': y_test7},
    'test8': {'x_cont': x_test8, 'y': y_test8},
    'test9': {'x_cont': x_test9, 'y': y_test9},
    'train_NoShuffle' : {'x_cont': X_train.astype(np.float32), 'y': y_train.astype(np.float32)}  
}

In [13]:
# Advanced preprocessing strategy.
# The noise is added to improve the output of QuantileTransformer in some cases.
X_cont_train_numpy = data_numpy['train']['x_cont']  
noise = (
    np.random.default_rng(0)
    .normal(0.0, 1e-5, X_cont_train_numpy.shape)
    .astype(X_cont_train_numpy.dtype)
)
preprocessing = sklearn.preprocessing.QuantileTransformer(
    n_quantiles=max(min(len(train_idx) // 30, 1000), 10),
    output_distribution='normal',
    subsample=10**9,
).fit(X_cont_train_numpy + noise)
del X_cont_train_numpy

# Apply the preprocessing.  
for part in data_numpy:
    data_numpy[part]['x_cont'] = preprocessing.transform(data_numpy[part]['x_cont'])

In [15]:
# Label preprocessing.
class RegressionLabelStats(NamedTuple):
    mean: float
    std: float


Y_train = data_numpy['train']['y'].copy()  
# For regression tasks, it is highly recommended to standardize the training labels.
regression_label_stats = RegressionLabelStats(
    Y_train.mean().item(), Y_train.std().item()
)
Y_train = (Y_train - regression_label_stats.mean) / regression_label_stats.std

print(Y_train)

[ 1.47296833e-02 -2.08790600e-01  3.19970191e-01 -8.97321761e-01
 -3.73284481e-02 -1.73576713e+00 -7.69691050e-01 -7.02039957e-01
 -1.15292335e+00  2.11113349e-01 -1.17188656e+00 -2.42163286e-01
 -2.72770345e-01  1.13488245e+00 -2.44799137e-01 -9.78310227e-01
  1.04783618e+00  2.38001649e-03  1.39437282e+00 -4.52812403e-01
  1.19878531e+00  5.40749073e-01  9.34183955e-01  4.95556146e-01
  2.37764931e+00  2.77974516e-01  6.98135316e-01 -3.38434100e-01
  1.51876703e-01  1.49097514e+00 -6.51740376e-03 -7.55352378e-01
  1.23194981e+00 -2.22039849e-01 -1.97223341e+00  1.07383645e+00
  2.90196687e-01 -1.40788841e+00  5.71168244e-01 -1.43598032e+00
 -4.24824387e-01 -1.80327189e+00 -3.71500701e-01 -2.09548056e-01
 -3.58117044e-01  1.64553344e+00 -3.39338891e-02 -1.86352767e-02
 -4.56062019e-01  7.95438886e-01 -9.75647092e-01 -1.06301558e+00
  4.74287122e-01 -8.06098759e-01 -4.97734457e-01 -1.48660696e+00
 -5.91463484e-02 -1.59293985e+00  9.62528706e-01 -4.96124886e-02
  5.49434364e-01 -1.20334

## PyTorch settings


In [16]:
# Device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

# Convert data to tensors
data = {
    part: {k: torch.as_tensor(v, device=device, dtype = torch.bfloat16) for k, v in data_numpy[part].items()}
    for part in data_numpy
}
Y_train = torch.as_tensor(Y_train, device=device)

for part in data:
    data[part]['y'] = data[part]['y'].float()
Y_train = Y_train.float()

# Automatic mixed precision (AMP)
# torch.float16 is implemented for completeness,
# but it was not tested in the project,
# so torch.bfloat16 is used by default.
amp_dtype = (
    torch.bfloat16
    if torch.cuda.is_available() and torch.cuda.is_bf16_supported()
    else torch.float16
    if torch.cuda.is_available()
    else None
)
# Changing False to True will result in faster training on compatible hardware.
amp_enabled = True and amp_dtype is not None
grad_scaler = torch.cuda.amp.GradScaler() if amp_dtype is torch.float16 else None  # type: ignore

# torch.compile
compile_model = False

# fmt: off
print(
    f'Device:        {device.type.upper()}'
    f'\nAMP:           {amp_enabled} (dtype: {amp_dtype})'
    f'\ntorch.compile: {compile_model}'
)
# fmt: on

cuda:0
Device:        CUDA
AMP:           True (dtype: torch.bfloat16)
torch.compile: False


In [17]:
print(amp_dtype)

torch.bfloat16


## Model


In [138]:
cat_cardinalities = []
n_classes = None

# Choose one of the two configurations below.

# TabM
#arch_type = 'tabm'
#arch_type = 'tabm-normal'
arch_type = 'tabm-mini-normal'
bins = None

# TabM-mini with the piecewise-linear embeddings.
#arch_type = 'tabm-mini'
#bins = rtdl_num_embeddings.compute_bins(data['train']['x_cont'])

model = Model(
    n_num_features=n_cont_features,
    cat_cardinalities=cat_cardinalities,
    n_classes=n_classes,
    backbone={
        'type': 'MLP',
        'n_blocks': 3 if bins is None else 2,
        'd_block': 512, #768, #512,
        'dropout': 0.2, #0.1,
    },
    bins=bins,
    num_embeddings=(
        None
        if bins is None
        else {
            'type': 'PiecewiseLinearEmbeddings',
            'd_embedding': 16, #16,
            'activation': False,
            'version': 'B',
        }
    ),
    arch_type=arch_type,
    k=64, #32,
    share_training_batches=True,
).to(device)
optimizer = torch.optim.AdamW(make_parameter_groups(model), lr=2e-3, weight_decay=3e-4)

if compile_model:
    # NOTE
    # `torch.compile` is intentionally called without the `mode` argument
    # (mode="reduce-overhead" caused issues during training with torch==2.0.1).
    model = torch.compile(model)
    evaluation_mode = torch.no_grad
else:
    evaluation_mode = torch.inference_mode

In [139]:
@torch.autocast(device.type, enabled=amp_enabled, dtype=amp_dtype)  # type: ignore[code]
def apply_model(part: str, idx: Tensor) -> Tensor:
    return (
        model(
            data[part]['x_cont'][idx],
            None,
        )
        .squeeze(-1)  # Remove the last dimension for regression tasks.
        .float()
    )

In [140]:
base_loss_fn = F.mse_loss

In [141]:
def loss_fn(y_pred: Tensor, y_true: Tensor) -> Tensor:
    # TabM produces k predictions. Each of them must be trained separately.
    # (regression)     y_pred.shape == (batch_size, k)
    # (classification) y_pred.shape == (batch_size, k, n_classes)
    k = y_pred.shape[-1]   #[-1 if task_type == 'regression' else -2]
    return base_loss_fn(
        y_pred.flatten(0, 1),
        y_true.repeat_interleave(k) if model.share_training_batches else y_true,
    )

In [142]:
@evaluation_mode()
def evaluate(part: str) -> float:
    model.eval()

    # When using torch.compile, you may need to reduce the evaluation batch size.
    eval_batch_size = 2 #8096
    y_pred: np.ndarray = (
        torch.cat(
            [
                apply_model(part, idx)
                for idx in torch.arange(len(data[part]['y']), device=device).split(
                    eval_batch_size
                )
            ]
        )
        .cpu()
        .numpy()
    )
    #  Transform the predictions back to the original label space.
    assert regression_label_stats is not None
    y_pred = y_pred * regression_label_stats.std + regression_label_stats.mean
    y_pred = y_pred.mean(1)

    y_true = data[part]['y'].cpu().numpy()
    score = (
        -(sklearn.metrics.mean_squared_error(y_true, y_pred) ** 0.5)
    )
    return float(score)  # The higher -- the better.

#print(type(data["val"]['x_cont']))
#print(data["test"]['x_cont'])
#print(data["test1"]['x_cont'])
#print(data["test2"]['x_cont'])
#print(data["test3"]['x_cont'])
#print(data["test4"]['x_cont'])

print(f'Val MSE score before training: {evaluate("val"):.4f}') 
print(f'Test MSE score before training: {evaluate("test"):.4f}') 
print(f'Test 1  MSE score before training: {evaluate("test1"):.4f}')  
print(f'Test 2  MSE score before training: {evaluate("test2"):.4f}')  
print(f'Test 3  MSE score before training: {evaluate("test3"):.4f}')  
print(f'Test 4  MSE score before training: {evaluate("test4"):.4f}')  
print(f'Test 5  MSE score before training: {evaluate("test5"):.4f}')  
print(f'Test 6  MSE score before training: {evaluate("test6"):.4f}') 
print(f'Test 7  MSE score before training: {evaluate("test7"):.4f}')  
print(f'Test 8  MSE score before training: {evaluate("test8"):.4f}')  
print(f'Test 9  MSE score before training: {evaluate("test9"):.4f}') 

Val MSE score before training: -76.8718
Test MSE score before training: -66.6881
Test 1  MSE score before training: -49.1727
Test 2  MSE score before training: -65.4859
Test 3  MSE score before training: -100.2274
Test 4  MSE score before training: -23.3664
Test 5  MSE score before training: -19.3730
Test 6  MSE score before training: -44.8747
Test 7  MSE score before training: -82.2445
Test 8  MSE score before training: -111.6298
Test 9  MSE score before training: -33.3674


## Training


In [143]:
import time, datetime
from datetime import datetime
from datetime import timedelta 

In [144]:
best = {
    'val': -math.inf,
    'test': -math.inf,
    'epoch': -1,
}

trainloss_list = []
valloss_list = []
testloss_list = []
testloss1_list = []
testloss2_list = []
testloss3_list = []
testloss4_list = []
testloss5_list = []
testloss6_list = []
testloss7_list = []
testloss8_list = []
testloss9_list = []

In [145]:
# For demonstration purposes (fast training and bad performance),
# one can set smaller values:
# n_epochs = 20
# patience = 2
n_epochs = 400 #1_000_000_000
patience = 400  # 16

train_size = len(train_idx)
print("train_size: ", train_size)
batch_size = 2  # 256
epoch_size = math.ceil(train_size / batch_size)


# Early stopping: the training stops when
# there are more than `patience` consequtive bad updates.
# patience = 16
remaining_patience = patience

print('-' * 88 + '\n')
for epoch in range(n_epochs):
    batches = (
        torch.randperm(train_size, device=device).split(batch_size)
        if model.share_training_batches
        else [
            x.transpose(0, 1).flatten()
            for x in torch.rand((model.k, train_size), device=device)
            .argsort(dim=1)
            .split(batch_size, dim=1)
        ]
    )
    for batch_idx in tqdm(batches, desc=f'Epoch {epoch}'):
        model.train()
        optimizer.zero_grad()
        loss = loss_fn(apply_model('train', batch_idx), Y_train[batch_idx])
        #trainloss_list.append(loss.cpu())
        if grad_scaler is None:
            loss.backward()
            optimizer.step()
        else:
            grad_scaler.scale(loss).backward()  # type: ignore
            grad_scaler.step(optimizer)
            grad_scaler.update()

    val_score = evaluate('val'); valloss_list.append(val_score)  
    test_score = evaluate('test'); testloss_list.append(test_score)
    test1_score = evaluate("test1"); testloss1_list.append(test1_score)
    test2_score = evaluate("test2"); testloss2_list.append(test2_score)
    test3_score = evaluate("test3"); testloss3_list.append(test3_score)
    test4_score = evaluate("test4"); testloss4_list.append(test4_score)
    test5_score = evaluate("test5"); testloss5_list.append(test5_score)
    test6_score = evaluate("test6"); testloss6_list.append(test6_score)
    test7_score = evaluate("test7"); testloss7_list.append(test7_score)
    test8_score = evaluate("test8"); testloss8_list.append(test8_score)
    test9_score = evaluate("test9"); testloss9_list.append(test9_score)
    print(f'(val) {val_score:.4f} (test) {test_score:.4f} (test1) {test1_score:.4f} (test2) {test2_score:.4f} (test3) {test3_score:.4f} (test4) {test4_score:.4f} (test5) {test5_score:.4f} (test6) {test6_score:.4f} (test7) {test7_score:.4f} (test8) {test8_score:.4f} (test9) {test9_score:.4f}')

    if (test_score > best['test']):  #((val_score > best['val']) and (test_score > best['test'])):
        print('🌸 New best epoch! 🌸')
        best = {'val': val_score, 'test': test_score, 'epoch': epoch}
        remaining_patience = patience

        
        save_dir = r'D:\Data\NHRI\NHDF\best model'
        last_save_path = os.path.join(save_dir, f'epoch.{epoch:04d}-val_loss.{val_score:.4f}-test_loss.{test_score:.4f}.pth')    
        torch.save(model, last_save_path) 
    else:
        remaining_patience -= 1

    if remaining_patience < 0:
        break

    print()

print('\n\nResult:')
print(best)




# save train process  
t = time.localtime()
t.tm_hour, t.tm_min, t.tm_sec    
outtime = str(datetime.today().date()) + ' ' + str(t.tm_hour) + 'H' + str(t.tm_min) + 'M' 
csv_path = r'D:\Data\NHRI\NHDF\train_process\TabM_train_process ' + outtime + '.csv'
loss_hist = np.column_stack((valloss_list, testloss_list, testloss1_list, testloss2_list, testloss3_list, testloss4_list, testloss5_list, testloss6_list, testloss7_list, testloss8_list, testloss9_list))
np.savetxt(csv_path, loss_hist, delimiter=",", header="val_loss,test_loss,1st_loss,2nd_loss,3rd_loss,4th_loss,5th_loss,6th_loss,7th_loss,8th_loss,9th_loss", encoding='utf-8-sig', comments='')




train_size:  480
----------------------------------------------------------------------------------------



Epoch 0: 100%|██████████| 240/240 [00:00<00:00, 409.88it/s]


(val) -29.3309 (test) -29.6270 (test1) -23.8200 (test2) -25.1615 (test3) -35.9173 (test4) -16.4138 (test5) -14.3206 (test6) -14.5076 (test7) -28.1366 (test8) -51.8666 (test9) -35.2492
🌸 New best epoch! 🌸



Epoch 1: 100%|██████████| 240/240 [00:00<00:00, 545.18it/s]


(val) -33.9062 (test) -32.6899 (test1) -16.3741 (test2) -16.4334 (test3) -32.0772 (test4) -12.2240 (test5) -12.3891 (test6) -21.2287 (test7) -47.0814 (test8) -69.6360 (test9) -15.2039



...

(val) -5.5163 (test) -10.3231 (test1) -5.5391 (test2) -5.1414 (test3) -16.7148 (test4) -6.7124 (test5) -6.5829 (test6) -7.9056 (test7) -8.5397 (test8) -10.9645 (test9) -16.6902



Epoch 304: 100%|██████████| 240/240 [00:00<00:00, 571.46it/s]


(val) -6.5434 (test) -9.1687 (test1) -4.5014 (test2) -4.1710 (test3) -10.0356 (test4) -7.0835 (test5) -8.6459 (test6) -10.7854 (test7) -11.0478 (test8) -10.5270 (test9) -12.0041



Epoch 305:  26%|██▋       | 63/240 [00:00<00:00, 564.62it/s]


KeyboardInterrupt: 

## use best model to predict and show Metric


In [146]:
loaded_model = torch.load(r'D:\Data\NHRI\NHDF\best model\epoch.0127-val_loss.-8.3623-test_loss.-6.1919.pth')
print(loaded_model)
from torchsummary import summary
summary(loaded_model, input_size= (1, 10))

Model(
  (backbone): MLP(
    (blocks): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=10, out_features=512, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.2, inplace=False)
      )
      (1-2): 2 x Sequential(
        (0): Linear(in_features=512, out_features=512, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.2, inplace=False)
      )
    )
  )
  (minimal_ensemble_adapter): ScaleEnsemble()
  (output): NLinear()
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
     ScaleEnsemble-1               [-1, 64, 10]             640
            Linear-2              [-1, 64, 512]           5,632
              ReLU-3              [-1, 64, 512]               0
           Dropout-4              [-1, 64, 512]               0
            Linear-5              [-1, 64, 512]         262,656
              ReLU-6              [-1, 64, 512]               0
           Dropout-7          

C:\Users\Administrator\AppData\Local\Temp\ipykernel_13732\3439836195.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load(r'D:\Data\NHRI\小论文4\2、模型\1

In [147]:
# modify to best model
def lz_apply_model(part: str, idx: Tensor) -> Tensor:
    return (
        loaded_model(
            data[part]['x_cont'][idx],
            None,
        )
        .squeeze(-1)  # Remove the last dimension for regression tasks.
        .float()
    )

In [148]:
def safe_mape(y_true, y_pred, epsilon=1e-8, threshold=0.01):
    
    
    y_true = torch.tensor(y_true)
    y_pred = torch.tensor(y_pred)
    
    
    y_pred = y_pred.to(y_true.device)
    
   
    mask = torch.abs(y_true) > threshold
    
  
    if mask.sum() == 0:
        return torch.tensor(float('nan'), device=y_true.device)
    
    
    ape = torch.abs((y_true[mask] - y_pred[mask]) / (y_true[mask] + epsilon))
    
    return torch.mean(ape) * 100

In [149]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
def lz_evaluate(part: str):
    model.eval()

    # When using torch.compile, you may need to reduce the evaluation batch size.
    eval_batch_size = 2 #8096
    y_pred: np.ndarray = (
        torch.cat(
            [
                lz_apply_model(part, idx)
                for idx in torch.arange(len(data[part]['y']), device=device).split(
                    eval_batch_size
                )
            ]
        )
        .cpu()
        .detach()
        .numpy()
    )
     Transform the predictions back to the original label space.
    assert regression_label_stats is not None
    y_pred = y_pred * regression_label_stats.std + regression_label_stats.mean
    y_pred = y_pred.mean(1)
    #print("预测结果：") #****************************************************************
    for item in y_pred:
        print(item)    #****************************************************************
    y_true = data[part]['y'].cpu().numpy()
    loss_RMSE = (
        (sklearn.metrics.mean_squared_error(y_true, y_pred) ** 0.5)
    )
    loss_MSE = (
        sklearn.metrics.mean_squared_error(y_true, y_pred)
    )
    loss_r2 = r2_score(y_true, y_pred)
    loss_MAPE = safe_mape(y_true, y_pred).item()
    loss_MAE = mean_absolute_error(y_true, y_pred)
    print("")
    print("Mean Absolute Error (MAE):", loss_MAE)
    print("Mean Squared Error (MSE):", loss_MSE)
    print("Root Mean Squared Error (RMSE):", loss_RMSE)
    print("Mean Absolute Percentage Error (MAPE):", loss_MAPE, "%")
    print("R² Score:", loss_r2)
    
    #return float(score)  # The higher -- the better.

In [150]:
# NOTE Train set
#print("X_train.shape: ", X_train.shape)
#print("y_train.shape: ", y_train.shape)
test_score = lz_evaluate('train_NoShuffle')

预测结果：
85.64632
98.03724
110.00989
119.89528
132.15063
144.01724
144.57852
160.52216
184.52487
198.20572
222.32791
244.89728
173.29529
199.37897
230.10779
249.91597
282.89395
306.48206
178.7734
206.68222
238.97043
258.3214
293.5077
316.2798
175.21439
199.41849
229.28777
249.54831
283.3249
308.90744
165.58221
181.8068
207.54994
225.7768
259.18155
285.70172
137.26419
154.16646
171.76605
183.3872
205.60667
224.83235
96.836205
104.65546
118.51181
127.91703
143.49481
153.11557
39.3036
50.039722
54.82646
62.39764
69.78088
76.09553
66.098526
87.99702
100.17361
111.26866
125.7166
137.78238
85.753586
128.92426
147.17435
161.64774
185.42929
205.65482
96.735756
143.65591
167.47473
181.457
209.3357
225.18852
92.792404
136.96869
158.08551
173.1279
188.76077
218.9763
84.629395
120.7236
135.9196
153.1753
165.98907
197.07191
67.388336
90.75149
104.32053
115.76603
127.79939
143.20554
36.171635
51.160408
58.18911
66.066956
72.4072
82.81189
11.729156
16.95976
23.205475
25.866152
28.243626
34.229965
24.006

In [152]:
# NOTE all test set
test_score = lz_evaluate('test')

预测结果：
61.13736
102.63955
129.52959
142.34534
140.9011
129.05096
102.89408
63.666485
44.95141
79.298706
109.06973
126.956024
125.09858
111.05064
87.17853
49.307335
26.738136
43.161
62.921867
69.93313
64.01273
53.93451
40.342712
23.156456
125.7244
141.34152
148.82257
139.75272
160.68315
149.71007
122.52712
112.70517
136.83783
154.30446
165.70546
151.57639
182.245
169.04694
134.08334
124.037025
169.11841
185.12332
205.08711
193.61093
207.33636
196.75003
170.62215
155.63135
208.2771
240.34721
258.65283
268.0894
260.00183
239.1582
201.55968
158.1178
189.82066
264.79248
289.9077
305.05005
289.24023
268.08463
210.83902
163.52206
142.984
181.32599
202.99886
199.8607
210.82718
198.46277
152.8688
128.80356

Mean Absolute Error (MAE): 4.889408588409424
Mean Squared Error (MSE): 38.35129928588867
Root Mean Squared Error (RMSE): 6.192842585266371
Mean Absolute Percentage Error (MAPE): 3.4640841484069824 %
R² Score: 0.9913492202758789


In [153]:
# NOTE 1st test set
test1_score = lz_evaluate("test1");

预测结果：
61.13736
102.63955
129.52959
142.34534
140.9011
129.05096
102.89408
63.666485

Mean Absolute Error (MAE): 4.5898118019104
Mean Squared Error (MSE): 23.654468536376953
Root Mean Squared Error (RMSE): 4.863585975016475
Mean Absolute Percentage Error (MAPE): 4.472901821136475 %
R² Score: 0.9683841466903687


In [154]:
# NOTE 2nd test set
test2_score = lz_evaluate("test2");

预测结果：
44.95141
79.298706
109.06973
126.956024
125.09858
111.05064
87.17853
49.307335

Mean Absolute Error (MAE): 1.3829421997070312
Mean Squared Error (MSE): 3.349565029144287
Root Mean Squared Error (RMSE): 1.830181692932231
Mean Absolute Percentage Error (MAPE): 1.5848506689071655 %
R² Score: 0.9961138963699341


In [155]:
# NOTE 3rd test set
test3_score = lz_evaluate("test3");

预测结果：
26.738136
43.161
62.921867
69.93313
64.01273
53.93451
40.342712
23.156456

Mean Absolute Error (MAE): 2.7796711921691895
Mean Squared Error (MSE): 9.629571914672852
Root Mean Squared Error (RMSE): 3.103155154785666
Mean Absolute Percentage Error (MAPE): 5.770124435424805 %
R² Score: 0.9678148627281189


In [156]:
# NOTE 4th test set
test4_score = lz_evaluate("test4")

预测结果：
125.7244
141.34152
148.82257
139.75272
160.68315
149.71007
122.52712
112.70517

Mean Absolute Error (MAE): 4.527172088623047
Mean Squared Error (MSE): 31.417938232421875
Root Mean Squared Error (RMSE): 5.605170669339326
Mean Absolute Percentage Error (MAPE): 3.577054023742676 %
R² Score: 0.9028952717781067


In [157]:
# NOTE 5th test set
test5_score = lz_evaluate("test5")

预测结果：
136.83783
154.30446
165.70546
151.57639
182.245
169.04694
134.08334
124.037025

Mean Absolute Error (MAE): 2.907179832458496
Mean Squared Error (MSE): 19.053531646728516
Root Mean Squared Error (RMSE): 4.365035125486222
Mean Absolute Percentage Error (MAPE): 1.8730480670928955 %
R² Score: 0.9438543915748596


In [158]:
# NOTE  6th test set
test6_score = lz_evaluate("test6")

预测结果：
169.11841
185.12332
205.08711
193.61093
207.33636
196.75003
170.62215
155.63135

Mean Absolute Error (MAE): 5.560386657714844
Mean Squared Error (MSE): 41.17732620239258
Root Mean Squared Error (RMSE): 6.416956147769173
Mean Absolute Percentage Error (MAPE): 2.933687925338745 %
R² Score: 0.8876039981842041


In [159]:
# NOTE  7th test set
test7_score = lz_evaluate("test7")

预测结果：
208.2771
240.34721
258.65283
268.0894
260.00183
239.1582
201.55968
158.1178

Mean Absolute Error (MAE): 7.494426727294922
Mean Squared Error (MSE): 73.00941467285156
Root Mean Squared Error (RMSE): 8.544554679610375
Mean Absolute Percentage Error (MAPE): 3.2136826515197754 %
R² Score: 0.9271987676620483


In [160]:
# NOTE  8th test set
test8_score = lz_evaluate("test8")

预测结果：
189.82066
264.79248
289.9077
305.05005
289.24023
268.08463
210.83902
163.52206

Mean Absolute Error (MAE): 5.397302627563477
Mean Squared Error (MSE): 48.87992477416992
Root Mean Squared Error (RMSE): 6.991417937312139
Mean Absolute Percentage Error (MAPE): 2.319110631942749 %
R² Score: 0.9824187755584717


In [161]:
# NOTE  9th test set
test9_score = lz_evaluate("test9")

预测结果：
142.984
181.32599
202.99886
199.8607
210.82718
198.46277
152.8688
128.80356

Mean Absolute Error (MAE): 9.365781784057617
Mean Squared Error (MSE): 94.98994445800781
Root Mean Squared Error (RMSE): 9.746278492738027
Mean Absolute Percentage Error (MAPE): 5.4322991371154785 %
R² Score: 0.8367069959640503
